## Set Up
In this first cell, we'll load the necessary libraries.

In [1]:
import tensorflow as tf 

print(tf.__version__)
tf.logging.set_verbosity(tf.logging.INFO)
# pd.options.display.max_rows = 10
# pd.options.display.float_format = '{:.1f}'.format

ModuleNotFoundError: No module named 'tensorflow'